In [ ]:
import numpy as np
import random
from ortools.linear_solver import pywraplp
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    # Calculate the great-circle distance between two points (in km)
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

class MCFLPD_3SH:
    def __init__(self, demand_points, facility_locations, p, num_drones, battery_range_km, facility_capacity):
        self.demand_points = demand_points  # list of tuples (lat, lon, demand)
        self.facility_locations = facility_locations  # list of tuples (lat, lon)
        self.p = p  # number of facilities to open
        self.num_drones = num_drones
        self.battery_range_km = battery_range_km
        self.facility_capacity = facility_capacity
        self.y_solution = None
        self.x_solution = None
        self.drone_assignments = None
        self.drones_per_facility = None

        # Calculate distance matrix between demand points and facilities
        self.dist_matrix = np.zeros((len(demand_points), len(facility_locations)))
        for i, (dlat, dlon, _) in enumerate(demand_points):
            for j, (flat, flon) in enumerate(facility_locations):
                self.dist_matrix[i, j] = haversine_distance(dlat, dlon, flat, flon)

    # STAGE 1: Facility Location and Allocation
    def solve_stage1(self):
        solver = pywraplp.Solver.CreateSolver('SCIP')
        if not solver:
            print("Solver not available.")
            return False

        m = len(self.demand_points)
        n = len(self.facility_locations)

        # Decision variables
        x = {}  # x[i,j] = 1 if demand i assigned to facility j
        y = {}  # y[j] = 1 if facility j is opened

        for i in range(m):
            for j in range(n):
                x[i, j] = solver.BoolVar(f'x_{i}_{j}')
        for j in range(n):
            y[j] = solver.BoolVar(f'y_{j}')

        # Objective: Maximize weighted demand served (Eq. 10)
        # Using b_ij = 1/dist_matrix[i,j] as weight (can be modified)
        solver.Maximize(solver.Sum(
            (self.demand_points[i][2] / max(1, self.dist_matrix[i, j])) * x[i, j]
            for i in range(m) for j in range(n)
        ))

        # Constraints
        # Each demand assigned to at most one facility within range (Eq. 11)
        for i in range(m):
            # Get facilities within drone range for this demand point
            valid_facilities = [j for j in range(n) if self.dist_matrix[i, j] <= self.battery_range_km]
            solver.Add(solver.Sum(x[i, j] for j in valid_facilities) <= 1)

        # Open exactly p facilities (Eq. 12)
        solver.Add(solver.Sum(y[j] for j in range(n)) == self.p)

        # Facility capacity constraint (Eq. 13)
        for j in range(n):
            solver.Add(solver.Sum(
                self.demand_points[i][2] * x[i, j] for i in range(m)
            ) <= self.facility_capacity)

        # Assign only to open facilities (Eq. 14)
        for i in range(m):
            for j in range(n):
                solver.Add(x[i, j] <= y[j])

        status = solver.Solve()

        if status == pywraplp.Solver.OPTIMAL:
            self.y_solution = [int(y[j].solution_value()) for j in range(n)]
            self.x_solution = [[int(x[i, j].solution_value()) for j in range(n)] for i in range(m)]
            print("Stage 1 solved: Facility locations decided.")
            return True
        else:
            print("No optimal solution found for stage 1.")
            self.y_solution = None
            self.x_solution = None
            return False

    # STAGE 2: Repeated Knapsack Problems for Drone Allocation
    def solve_stage2(self):
        if self.y_solution is None:
            print("Stage 1 not solved yet.")
            return 0

        open_facilities = [j for j, val in enumerate(self.y_solution) if val == 1]
        self.drones_per_facility = [0] * len(self.facility_locations)
        self.drone_assignments = {}  # {drone_id: (facility, [demand_points])}
        
        remaining_demand = {}
        for j in open_facilities:
            # Get demand points assigned to this facility in stage 1
            remaining_demand[j] = [i for i in range(len(self.demand_points)) 
                                 if self.x_solution[i][j] == 1]
        
        total_served_demand = 0
        drone_id = 0
        
        while drone_id < self.num_drones and any(remaining_demand.values()):
            best_facility = None
            best_demand = []
            best_value = 0
            
            # Solve knapsack for each facility with remaining demand
            for j in open_facilities:
                if not remaining_demand[j]:
                    continue
                    
                # Solve knapsack problem for this facility
                demand_indices = remaining_demand[j]
                demands = [self.demand_points[i][2] for i in demand_indices]
                distances = [self.dist_matrix[i, j] for i in demand_indices]
                
                # Simple heuristic: take demands until battery runs out
                sorted_indices = sorted(range(len(demand_indices)), 
                                 key=lambda i: demands[i]/distances[i], 
                                 reverse=True)
                
                current_load = 0
                current_distance = 0
                selected = []
                
                for idx in sorted_indices:
                    i = demand_indices[idx]
                    dist = distances[idx]
                    demand = demands[idx]
                    
                    # Check if adding this demand would exceed battery range
                    # (Assuming round trip distance)
                    if current_distance + 2*dist <= self.battery_range_km:
                        selected.append(i)
                        current_load += demand
                        current_distance += 2*dist
                
                # Track the best solution across all facilities
                if current_load > best_value:
                    best_value = current_load
                    best_facility = j
                    best_demand = selected
            
            if best_facility is not None:
                # Assign drone to this facility with these demands
                self.drones_per_facility[best_facility] += 1
                self.drone_assignments[drone_id] = (best_facility, best_demand)
                total_served_demand += best_value
                
                # Remove served demands
                remaining_demand[best_facility] = [
                    i for i in remaining_demand[best_facility] 
                    if i not in best_demand
                ]
                
                drone_id += 1
            else:
                break
        
        print(f"Stage 2 solved: Total demand served = {total_served_demand}")
        print(f"Drone allocation: {self.drones_per_facility}")
        return total_served_demand

    # STAGE 3: r-Exchange Heuristic
    def solve_stage3(self, r=1, max_iter=100):
        if self.y_solution is None:
            print("Stage 1 not solved yet.")
            return 0

        current_solution = [j for j, val in enumerate(self.y_solution) if val == 1]
        best_solution = current_solution.copy()
        best_demand = self.evaluate_solution(current_solution)
        
        for iteration in range(max_iter):
            # Calculate facility demands
            facility_demands = self.calculate_facility_demands(current_solution)
            
            # Identify r worst facilities (lowest demand)
            worst_indices = np.argsort(facility_demands)[:r]
            worst_facilities = [current_solution[i] for i in worst_indices]
            
            # Get candidate facilities (not currently open)
            candidate_facilities = [j for j in range(len(self.facility_locations)) 
                                  if j not in current_solution]
            
            if not candidate_facilities or len(worst_facilities) == 0:
                break
            
            # Create new solution by replacing worst facilities with random candidates
            new_solution = current_solution.copy()
            for f in worst_facilities:
                new_solution.remove(f)
            
            new_candidates = random.sample(candidate_facilities, min(r, len(candidate_facilities)))
            new_solution.extend(new_candidates)
            new_solution = list(set(new_solution))  # Remove duplicates
            
            if len(new_solution) != self.p:
                continue
            
            # Evaluate new solution
            current_demand = self.evaluate_solution(new_solution)
            
            # Update best solution if improved
            if current_demand > best_demand:
                best_solution = new_solution.copy()
                best_demand = current_demand
                current_solution = new_solution.copy()
                print(f"Iteration {iteration}: Improved solution found with demand {best_demand}")
        
        # Update the final solution
        self.y_solution = [1 if j in best_solution else 0 for j in range(len(self.facility_locations))]
        self.solve_stage1()  # Reoptimize assignments
        final_demand = self.solve_stage2()  # Reallocate drones
        
        print(f"Stage 3 completed. Best demand served: {final_demand}")
        return final_demand

    def evaluate_solution(self, solution):
        """Evaluate total demand served by a given facility solution"""
        # Temporary assignment to evaluate
        temp_y = [1 if j in solution else 0 for j in range(len(self.facility_locations))]
        
        # Solve assignment problem with fixed facilities
        solver = pywraplp.Solver.CreateSolver('SCIP')
        m = len(self.demand_points)
        n = len(self.facility_locations)
        
        x = {}
        for i in range(m):
            for j in range(n):
                x[i, j] = solver.BoolVar(f'x_{i}_{j}')
        
        # Objective: Maximize demand served
        solver.Maximize(solver.Sum(
            self.demand_points[i][2] * x[i, j]
            for i in range(m) for j in range(n)
            if temp_y[j] == 1 and self.dist_matrix[i, j] <= self.battery_range_km
        ))
        
        # Constraints
        for i in range(m):
            valid_facilities = [j for j in range(n) 
                             if temp_y[j] == 1 and self.dist_matrix[i, j] <= self.battery_range_km]
            solver.Add(solver.Sum(x[i, j] for j in valid_facilities) <= 1)
        
        for j in range(n):
            if temp_y[j] == 1:
                solver.Add(solver.Sum(
                    self.demand_points[i][2] * x[i, j] for i in range(m)
                ) <= self.facility_capacity)
        
        status = solver.Solve()
        
        if status == pywraplp.Solver.OPTIMAL:
            # Get total demand served
            total_demand = sum(
                self.demand_points[i][2] * x[i, j].solution_value()
                for i in range(m) for j in range(n)
            )
            return total_demand
        return 0

    def calculate_facility_demands(self, solution):
        """Calculate demand served by each facility in current solution"""
        facility_demands = np.zeros(len(solution))
        temp_y = [1 if j in solution else 0 for j in range(len(self.facility_locations))]
        
        solver = pywraplp.Solver.CreateSolver('SCIP')
        m = len(self.demand_points)
        n = len(self.facility_locations)
        
        x = {}
        for i in range(m):
            for j in range(n):
                x[i, j] = solver.BoolVar(f'x_{i}_{j}')
        
        # Objective: Maximize demand served
        solver.Maximize(solver.Sum(
            self.demand_points[i][2] * x[i, j]
            for i in range(m) for j in range(n)
            if temp_y[j] == 1 and self.dist_matrix[i, j] <= self.battery_range_km
        ))
        
        # Constraints
        for i in range(m):
            valid_facilities = [j for j in range(n) 
                             if temp_y[j] == 1 and self.dist_matrix[i, j] <= self.battery_range_km]
            solver.Add(solver.Sum(x[i, j] for j in valid_facilities) <= 1)
        
        status = solver.Solve()
        
        if status == pywraplp.Solver.OPTIMAL:
            for pos, j in enumerate(solution):
                facility_demands[pos] = sum(
                    self.demand_points[i][2] * x[i, j].solution_value()
                    for i in range(m)
                )
        
        return facility_demands

    def print_solution(self):
        if self.y_solution is None or self.x_solution is None:
            print("No solution available.")
            return

        print("\n=== Facility Location Solution ===")
        print(f"Facilities opened at locations (indices): {[i for i, v in enumerate(self.y_solution) if v == 1]}")
        print(f"Drones allocated per facility: {self.drones_per_facility}")
        
        print("\n=== Demand Assignments ===")
        total_demand = 0
        for i, assignments in enumerate(self.x_solution):
            for j, assigned in enumerate(assignments):
                if assigned == 1:
                    demand = self.demand_points[i][2]
                    dist = self.dist_matrix[i, j]
                    print(f"  Demand point {i} (demand: {demand}) -> Facility {j} (distance: {dist:.2f}km)")
                    total_demand += demand
                    
        print(f"\nTotal demand served: {total_demand}")
        
        if self.drone_assignments:
            print("\n=== Drone Assignments ===")
            for drone_id, (facility, demands) in self.drone_assignments.items():
                print(f"Drone {drone_id} at facility {facility} serves:")
                for i in demands:
                    demand = self.demand_points[i][2]
                    dist = self.dist_matrix[i, facility]
                    print(f"  - Demand point {i} (demand: {demand}, distance: {dist:.2f}km)")